In [1]:

from pandas import DataFrame as df
from scipy import ndimage as ndi
import skimage
from skimage import data, img_as_float
from skimage import io
from skimage.filters import threshold_triangle, gaussian
from skimage.segmentation import watershed
from skimage.measure import regionprops
import numpy as np

In [2]:
loc_l = 'images/01/'
loc_u = '.tif'
centroid_x = []
centroid_y = []
object_id = []
Timepoint = []
for i in range(1,93):
    loc = loc_l + str(i) + loc_u
    img = io.imread(loc)
    img = img_as_float(img)
    img = gaussian(img)
    thresh = skimage.filters.threshold_triangle(img)
    binary = img > thresh
    distance = ndi.distance_transform_edt(binary)
    local_maxi = np.copy(distance)
    local_maxi = local_maxi>(np.max(local_maxi)*0.3)
    coordinates = np.where(local_maxi)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=binary)
    props = regionprops(labels)
    for prop in props:
        Timepoint.append(i-1)
        y0,x0 = prop.centroid
        centroid_x.append(x0)
        centroid_y.append(y0)
object_id = [i for i in range(1,len(centroid_x)+1)]
Track_id = [0 for i in range(1,len(centroid_x)+1)]

C:\Users\nvkkr\Anaconda3\lib\site-packages\skimage\external\tifffile\tifffile.py:2616: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


In [3]:
total_data = [object_id,centroid_x,centroid_y,Timepoint,Track_id]

In [4]:
data = df(object_id,columns = ['object_ID'])
data['Centroid_X'] = centroid_x
data['Centroid_Y'] = centroid_y
data['TimePoint'] = Timepoint
data['Track_ID'] = Track_id
data

,object_ID,Centroid_X,Centroid_Y,TimePoint,Track_ID
0,1,568.842088,43.523290,0,0
1,2,193.539445,105.115999,0,0
2,3,827.703926,182.583758,0,0
3,4,79.103177,203.846771,0,0
4,5,398.584257,306.004157,0,0
...,...,...,...,...,...
2003,2004,711.696280,724.280664,91,0
2004,2005,457.065605,787.163823,91,0
2005,2006,430.870270,794.583784,91,0
2006,2007,740.425177,955.945408,91,0


In [5]:
import csv
import io

In [8]:
data.to_csv("./images/coordinates/experiment_1.csv")